In [19]:
import requests 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime,timedelta
import pytz

In [20]:
API_KEY='1afdd88fb14c4b25e2e9192d7eabbba9'
BASE_URL='https://api.openweathermap.org/data/2.5/'

In [21]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response=requests.get(url)
    data=response.json()
    return {
        'city':data['name'],
        'current_temp':round(data['main']['feels_like']),
        'feels_like':round(data['main']['feels_like']),
        'temp_min':round(data['main']['temp_min']),
        'temp_max':round(data['main']['temp_max']),
        'humidity':round(data['main']['humidity']),
        'description':data['weather'][0]['description'],
        'country':data['sys']['country'],
        'wind_gust_dir':data['wind']['deg'],
        'pressure':data['main']['pressure'],
        'WindGustSpeed':data['wind']['speed']
    }

In [22]:
import pandas as pd

def read_historical_data(filename):
    df = pd.read_csv(filename)  # Read CSV file
    df = df.dropna()  # Remove rows with missing values
    df = df.drop_duplicates()  # ✅ Corrected: Drop duplicate rows

    return df


In [33]:
from sklearn.preprocessing import LabelEncoder

def prepare_data(data):
    le = LabelEncoder() 
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

    X = data[['MinTemp', 'MaxTemp', 'WindGustDir','WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]
    y = data['RainTomorrow']

    return X, y, le


In [34]:
def train_test_model(X,y):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
    model=RandomForestClassifier(n_estimators=100,random_state=42)
    model.fit(X_train,y_train)
    prediction=model.predict(X_test)
    print('Mean squared error for rain model')
    print(mean_squared_error(y_test,prediction))
    return model

In [46]:
def prepare_regression_data(data,feature):
    X,y=[],[]

    for i in range(len(data)-1):
        X.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i+1])
    
    X=np.array(X).reshape(-1,1)
    y=np.array(y)
    return X,y


In [47]:
def train_regression_mode(X,y):
    model=RandomForestRegressor(n_estimators=100,random_state=42)
    model.fit(X,y)
    return model

In [48]:
def predict_future(model,current_value):
    predictions=[current_value]
    for i in range(5):
        next_value=model.predict(np.array([[predictions[-1]]]))
        predictions.append(next_value[0])
        
    return predictions[1:]

In [55]:


def weather_view():
    city=input("Enter any city name : ")
    current_weather= get_current_weather(city)
    historical_data = read_historical_data(r'C:\Users\Srushti\Downloads\weather.csv')
    X,y,le=prepare_data(historical_data)
    rain_model=train_test_model(X,y)
    wind_deg=current_weather['wind_gust_dir']%360
    compass_points=[
        ("N",0,11.25),("NNE",11.25,33.75),("NE",33.75,56.25),
        ("ENE",56.25,78.75),("E",78.75,101.25),("ESE",101.25,123.75),
        ("SE",123.75,146.25),("SSE",146.25,168.75),("S",168.75,191.25),
        ("SSW",191.25,213.75),("SW",213.75,236.25),("WSW",236.25,258.75),
        ("W",258.75,281.25),("WW",281.25,303.75),("NW",303.75,326.25),
        ("NW",326.25,348.75)
    ]
    compass_direction=next(point for point,start,end in compass_points if start<=wind_deg <end)
    compass_direction_encoded=le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    current_data={
        'MinTemp':current_weather['temp_min'],
        'MaxTemp':current_weather['temp_max'],
        'WindGustDir':compass_direction_encoded,
        'WindGustSpeed': current_weather['WindGustSpeed'],  # ✅ Correct key
        'Humidity':current_weather['humidity'],
        'Pressure':current_weather['pressure'],
        'Temp':current_weather['current_temp'],
    }
    current_df=pd.DataFrame([current_data])

    rain_prediction=rain_model.predict(current_df)[0]
    X_temp,y_temp=prepare_regression_data(historical_data,'Temp')
    X_hum,y_hum=prepare_regression_data(historical_data,'Humidity')
    temp_model=train_regression_mode(X_temp,y_temp)
    hum_mode=train_regression_mode(X_hum,y_hum)
    future_temp=predict_future(temp_model,current_weather['temp_min'])
    future_humidity=predict_future(hum_mode,current_weather['humidity'])

    timezone=pytz.timezone('Asia/Karachi')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour=next_hour.replace(minute=0,second=0,microsecond=0)

    future_times=[(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    print(f"City: {city},{current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}")
    print(f"Feels like: {current_weather['feels_like']}")
    print(f"Minimum Temperature: {current_weather['temp_min']}")
    print(f"MaximumTemperature: {current_weather['temp_max']}")
    print(f"Humidity:{current_weather['humidity']}%")
    print(f"Weather Prediction:{current_weather['description']}")
    print(f"Rain Prediction:{'Yes' if rain_prediction else 'No'}")

    print("Fututre temperature predicition")

    for time,temp in zip(future_times,future_temp):
        print(f"{time}:{round(temp,1)}")

    print("future humidity prediction")

    for time,humidity in zip(future_times,future_humidity):
        print(f"{time}:{round(humidity,1)}%")
        

weather_view()
                  


Enter any city name :  Mumbai


Mean squared error for rain model
0.1506849315068493
City: Mumbai,IN
Current Temperature: 27
Feels like: 27
Minimum Temperature: 27
MaximumTemperature: 28
Humidity:36%
Weather Prediction:smoke
Rain Prediction:Yes
Fututre temperature predicition
19:00:28.1
20:00:25.1
21:00:20.1
22:00:16.6
23:00:14.4
future humidity prediction
19:00:47.3%
20:00:47.2%
21:00:47.2%
22:00:47.2%
23:00:47.2%


In [43]:
print(get_current_weather('India'))

{'city': 'Innichen', 'current_temp': 0, 'feels_like': 0, 'temp_min': 2, 'temp_max': 2, 'humidity': 76, 'description': 'overcast clouds', 'country': 'IT', 'wind_gust_dir': 310, 'pressure': 1020, 'WindGustSpeed': 2.06}
